In [2]:
#The parsing and analsyis of preprocessed data for Fig4_Fig5_general.ipynb in demo

import torch
import sys
import os
import io
import json
from os.path import exists
import pickle

import numpy as np
from scipy.special import softmax
from scipy.stats import ranksums, wilcoxon, kstest, ttest_ind
from scipy import stats
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

from dynamics.utils import utils
from dynamics.vis import dynamics as dyplot
from dynamics.process.rnn import wt_kindergarten, wt_nets, wt_costs, wt_reinforce_cont_new, wt_pred, parse
from dynamics.vis import wt_vis
from dynamics.analysis import wt_analysis as wta


#remove
#import glob
#import warnings
#import statsmodels.api as sm

%load_ext autoreload
%autoreload 2
#%matplotlib notebook
#%matplotlib widget 
#out = widgets.Output(layout = widgets.Layout(height='300px'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#path info

#change entry number k==27 to 20-100 to lookg at long training

kuse = [10,60,40,
        20,20,20,20,
        10,10, 
        40, 100, 40, 20, 20,
        40, 40, 20, 20, 
        0, 0, 0, 0, 0, 0, 0, 0,
       10, 10, 100, 
        10]  # the maximum index for the final stage 5 and stage 6 training stage, by s_idx

num_rnns = {0:50, 1:50, 2:20, 3:20, 4:10, 5:10, 6:10, 7:10, 8:10, 9:10, 10:10, 11:20,
           12:10, 13:10, 14:20, 15:20, 16:10, 17:10,
           26:10, 27:10, 28:5, 29:50}  # number of RNNs per training type, by sidx


d0 = '/scratch/dh148/dynamics/results/rnn/ac/'
# paths for different datasets
dbase = d0 + '20231003/'
dbase_sham = d0 + '20230915_d2m/'
dbase_long = d0 + '/20240525_nok_nocl_longsim/'
dbase0 = d0
dbase_onehot = d0 + '20240724_onehot/'
dbase_onehot_newreg = d0 + '20240724_onehot_newreg_1/'
dbase_generic_grid = d0 + '20240803_genkind_indlosskind/'
dbase_si = d0 + '20240821_shortinference/'
dbase_if = d0 + '20240820_inf_first/'
dbase_longtrain = d0 + '20240525_nok_nocl_longsim/'
dbasegru = d0 + '20241113_empire/'

# for figure 3
subdirlist = ['full_cl/', 'nok_cl/','nok_nocl/',
              'pkind_mem/', 'pkind_count/', 'pkind_int/', 'pkind_pred/',
             'wkind/','wokind/', 
              'nok_nocl_wreg/', 'nok_nocl_longsim/', 'pkind2_mem_inf/', 'gen_kind/','gen_kind_newreg/',
              'pkind3_mem_inf_count/','pkind3_mem_inf_int/', 'gen_kind_lowreg/','onehot/',
              'gen_kind_highreg/','genkind_indlosskind/','onehot_newreg/',
             'L50/', 'L100/','L500/','L1000/','L1000_mem100/',
             'inferencefirst/', 'shortinference/','nok_nocl_longsim/','gru/']

ddict = {'full_cl/':dbase, 'nok_cl/':dbase, 'nok_nocl/':dbase, #2
         'pkind_mem/':dbase, 'pkind_count/':dbase, 'pkind_int/':dbase, 'pkind_pred/':dbase, #6
         'wkind/':dbase_sham, 'wokind/':dbase_sham, 'nok_nocl_wreg/':dbase, 'nok_nocl_longsim/':dbase_long, #10
         'pkind2_mem_inf/':dbase, 'gen_kind/':dbase, 'gen_kind_newreg/':dbase, #13
         'pkind3_mem_inf_count/':dbase, 'pkind3_mem_inf_int/':dbase, 'gen_kind_lowreg/':dbase, #16
         'onehot/':dbase_onehot, 'gen_kind_highreg/':dbase, #18
          'genkind_indlosskind/':dbase, 'onehot_newreg/':dbase_onehot_newreg, #20
        'L50/':dbase_generic_grid, 'L100/':dbase_generic_grid,'L500/':dbase_generic_grid, #23
         'L1000/':dbase_generic_grid, 'L1000_mem100/':dbase_generic_grid, #25
        'inferencefirst/':dbase_if,'shortinference/':dbase_si, 'nok_nocl_longsim/':dbase_longtrain,
        'gru/':dbasegru} #28

datadir_dat = lambda s_idx: ddict[subdirlist[s_idx]]+subdirlist[s_idx]



# nameing code. before I made the utils.getfnames() code, and too afriad to replace


configdir = lambda s_idx: '/home/dh148/projects/dynamics/jobs/20231003_clstudy/rnn/'+subdirlist[s_idx]
suff = '.json' #file ending
    
fgen = lambda num,idx, base, sess, s_idx : datadir_dat(s_idx)+str(num)+'/'+base + str(num)+'_'+sess+'_'+str(idx)

fname_funs = [lambda num,idx, s_idx: datadir_dat(s_idx)+str(num)+'/'+'rnn_kindergarten_' + str(num)+'_simple',
              lambda num,idx, s_idx: datadir_dat(s_idx)+str(num)+'/'+'rnn_kindergarten_' + str(num)+'_int_0_'+str(idx),
              lambda num,idx, s_idx: datadir_dat(s_idx)+str(num)+'/'+'rnn_pred_' + str(num)+'_'+str(idx),
              lambda num,idx, s_idx: fgen(num, idx, 'rnn_curric_', 'nocatch', s_idx),
              lambda num,idx, s_idx: fgen(num, idx, 'rnn_curric_', 'catch', s_idx),
              lambda num,idx, s_idx: fgen(num, idx, 'rnn_curric_', 'block', s_idx),
              lambda num,idx, s_idx: fgen(num, idx, 'rnn_curric_', 'block', s_idx)+'_freeze'
             ]

configname_fun = lambda num, s_idx: configdir(s_idx) +str(num)+'.cfg'
fname_behdat_fun = lambda num,s_idx: datadir_dat(s_idx) + 'rnn_'+str(num)+'_allbeh.json'
fname_behdat_fun_1k = lambda num,s_idx: datadir_dat(s_idx) + 'rnn_'+str(num)+'_allbeh_1k.json'


In [7]:
# decide on figure

fig2do = '3'  # full, nok_cl, nok_nocl
#fig2do = 's3'  # single kindergarten
#fig2do = 's3_pkind2'  # not used
#fig2do = 's3_wreg'  # regularization only 
#fig2do = '5'  # sham tasks (one-hot)
#fig2do = 'pkind3'  # multi-task kinderaten
#fig2do = 'onehot'
#fig2do = 'gen_kind'  #generic kindergarten
#fig2do = 'inf_manip'  # inference manipulations (short inference, inference first)
#fig2do = 'longtrain'


savedir = '/scratch/dh148/dynamics/results/rnn/ac/20231003/figs/'

klist_dict = {'3': [0,1,2], 's3':[0,3,4,5,6], '5':[0,7,8], 's3_pkind2': [0,3,6,11], 's3_wreg': [0,1,9],
               'pkind3': [0,11,14,15],'gen_kind':[0,1,8,9,13], 'onehot':[0,1,17], 'inf_manip': [0,26, 27],
             'longtrain':[0,28]}


labs = [k.strip('/') for k in subdirlist]

labs_fig = np.array(labs)[klist_dict[fig2do]]

#select averaging function
uf = np.nanmean
#uf = np.nanmedian

klist = klist_dict[fig2do]


In [5]:
def retrieve_behdat(fname_behdat, fname):
    """ helper code to lookup the dat in the behavioral data.
    if fname_behda is a list, just search it (for faster lookup)
    if a filename, load the file."""
    
    fname_base = fname.split('/')[-1]
    dat = None
    
    if type(fname_behdat) is list:  #easier to recylce
        datlist = fname_behdat
    else:
        #datlist = pickle.load(open(fname_behdat,'rb'))
        datlist = json.load(open(fname_behdat,'r'))
 
    for j in datlist:
        if j['name'] == fname_base:
            dat = j
            break
            
    return dat

In [13]:
dat.keys()

dict_keys(['tmesh', 'state_reset_idx', 'ops', 'env', 'rewvols', 'iscatch', 'blocks', 'outcomes', 'wt', 'rewards_pertrial', 'reward_delay', 'w_all', 'loss', 'loss_aux', 'rewardrate_pergradstep', 'annealed_weights', 'grad_all', 'theta', 'actions', 'values', 'rewards', 'preds', 'outputs_supervised', 'pi', 'states', 'inputs'])

In [41]:
with open('/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/rnn_5_allbeh.json','r') as f:
    dat = json.load(f)
    
    
dat[0]['name']

'rnn_curric_5_block_20.json'

In [82]:
# choose a particular curriculum type to simulate, if needed
srun = 13
simulate = True

In [83]:
# aggregate reward at end of training

if simulate:

    rr_means = []
    rr_sems = []
    rr_sampsforstats = []

    dt = 0.05

    for k in [srun]:
    #for k in klist:
        print(k)

        ddir = dbase + subdirlist[k]
        rr = []
        if k > 1:
            fname_fun_dat_k = fname_funs[5]  #freeze index
        else:
            fname_fun_dat_k = fname_funs[5]  #not freeze index

        maxidx = num_rnns[k]  # max number of RNNs
        print(maxidx)

        for j in range(1,maxidx+1):
            print(j,end='\r')

            if k == 2 or k == 7 or k == 8:  # dont have full training data beh for no_cl, so used sampled behavior w frozen model
                fname_behdat = fname_behdat_fun_1k(j,k)
            else:
                fname_behdat = fname_behdat_fun(j,k)

            if exists(fname_behdat):

                if k == 2 or k == 7 or k == 8:
                    fname_dat = fname_fun_dat_k(j,kuse[k], k).split('/')[-1]+'_1k'+suff
                else:
                    fname_dat = fname_fun_dat_k(j,kuse[k], k).split('/')[-1]+suff
                print(fname_fun_dat_k(j,kuse[k], k))

                try:

                    dat_stat = retrieve_behdat(fname_behdat,fname_dat)
                    if dat_stat is not None:
                        #average reward over a gradient step of 500 timesteps divided by time, divided by optimal amount
                        #so reward rate (rew/s), divided by max reward per trial

                        rr.append(uf(dat_stat['rewardrate_pergradstep'])/dt)
                    else:
                        print('data didnt exist?')
                        print(fname_stat)
                except:
                    print('json error:' +str(j))
                    print(fname_dat)
            else:
                print('skipping beh file:'+fname_behdat)



        print(len(rr))
        rr_means.append(np.nanmean(rr))
        rr_sampsforstats.append(rr)
        rr_sems.append(np.std(rr)/np.sqrt(len(rr)))
else:
    print('skipping simulation')
    
    

13
10
skipping beh file:/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/rnn_1_allbeh.json
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/rnn_curric_5_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_block_20
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_curric_10_block_20
9


In [86]:
#calculate reward per gradient step. end of training comparison across CL types

if simulate:
    rr_means_all = []
    rr_sems_all = []
    dt = 0.05

    for s_idx in [srun]:

        #for each CL type
        rr_means_k = []
        rr_sems_k = []

        for tphase in [3,4,5]:
            #Vuse = Vusevec[n]
            fname_fun_dat_k = fname_funs[tphase]  #freeze index
            if tphase < 5:
                nmax = 11
            else:
                nmax = kuse[s_idx]+1
            for idx in range(1,nmax):

                rr_idx = []  # for each index

                #maxidx = num_rnns[k]  # max number of RNNs

                for num in range(1,num_rnns[s_idx]+1):
                    print([s_idx,tphase, idx, num],end='\r')
                    # TODO
                    if s_idx == 2 or s_idx == 7 or s_idx == 8:
                        fname_behdat = fname_behdat_fun_1k(num,s_idx)
                    else:
                        fname_behdat = fname_behdat_fun(num,s_idx)

                    if exists(fname_behdat):
                        if s_idx == 2 or s_idx == 7 or s_idx == 8:
                            fname_dat = fname_fun_dat_k(num,idx, s_idx).split('/')[-1]+'_1k'+suff
                        else:
                            fname_dat = fname_fun_dat_k(num,idx, s_idx).split('/')[-1]+suff

                        try:
                            dat_stat = retrieve_behdat(fname_behdat,fname_dat)
                            if dat_stat is not None:

                                rr_idx.append(uf(dat_stat['rewardrate_pergradstep'])/dt)
                                #rr_idx.append(uf(dat_stat['rewardrate_pergradstep'])/dt/Vuse)
                                #rr_idx.append(uf(dat_stat['rewards_pertrial'])/Vuse)
                            else:
                                print('missing file in behdat: '+fname_fun_dat_k(num,idx, s_idx))
                        except:
                            print('Json error: ' + str([s_idx,tphase,num,idx]))


                rr_means_k.append(uf(rr_idx))
                rr_sems_k.append(np.nanstd(rr_idx)/np.sqrt(len(rr_idx)))

                #print(rr_means_k)


        rr_means_all.append(rr_means_k)
        rr_sems_all.append(rr_sems_k)
else:
    print('skipping simulation')


missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/rnn_curric_5_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_nocatch_1
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_c

missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_nocatch_9
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_nocatch_9
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_nocatch_9
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_nocatch_9
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_curric_10_nocatch_9
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_nocatch_10
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3_nocatch_10
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_nocatch_10
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/r

missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/rnn_curric_5_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_catch_8
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_curric_10_catch_8

missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_block_6
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_block_6
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_curric_10_block_6
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_block_7
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3_block_7
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_block_7
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/rnn_curric_5_block_7
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_block_7
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_block_7

missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/4/rnn_curric_4_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/5/rnn_curric_5_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/6/rnn_curric_6_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/7/rnn_curric_7_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/8/rnn_curric_8_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/9/rnn_curric_9_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/10/rnn_curric_10_block_15
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/2/rnn_curric_2_block_16
missing file in behdat: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/3/rnn_curric_3

In [87]:
#SAVE
dosave = True
if dosave:
    print('saving')
    savedir = '/scratch/dh148/dynamics/results/rnn/ac/20231003/figs/'
    with open(savedir+'rrate_sidx_'+str(srun)+'.dat', 'wb') as f:
            pickle.dump([rr_means_all[0], rr_sems_all[0],rr_sampsforstats[0]],f)


saving


In [49]:
def beh_acrosstraining(s_idx = 0, num = 0, switchblock = False):
    """
    retrieves behavioral data for a single network across training 
    s_idx: CL type 0:full, 1:nok_cl, 2: nok_nocl, others are pkind?
    num: RNN number
    """
    
    stagelist = []
    idxlist = []
    
    #lists for block-relevant things
    stagelist_block = []
    idxlist_block = []
    
    #lin sensitivity, wt ratio, historical regression, opt out, violation
    sensitivity = []
    sensitivity_pval = []
    optout = []
    vio = []
    wt_ratio = []
    histreg_coeff = []
    histreg_pval = []
    rs_hist = []  #variance explained
    

    # load the bulk behavioral data
    fname_fun_stage = lambda stage, idx: fname_funs[stage](num, idx, s_idx)  # helper name fun for given RNN
    freezemax_idx = 0 # track the last freeze block sesion for each RNN. only plot this. 
    if s_idx == 2 or s_idx == 7 or s_idx == 8:
        fname_behdat = fname_behdat_fun_1k(num,s_idx)
    else:
        fname_behdat = fname_behdat_fun(num,s_idx)
    print(fname_behdat)
    
    if exists(fname_behdat):
        with open(fname_behdat,'r') as f:
            alldat_list = json.load(f)
    else:
        print('behdat does not exist. aborting: '+ fname_behdat)
        return None
    
    
    # ideally will all be same idx when trainign is complete
    idx_max = [0,0,0,10,10,kuse[s_idx]]  # max possible idx per training stage
    print(idx_max)
    for stage in [3,4,5]:
            
        idxrange = range(1,idx_max[stage]+1)  # number samples per training stage
        for idx in idxrange:
            print([stage,idx], end='\r')
            if s_idx == 2 or s_idx == 7 or s_idx == 8:
                fname = fname_fun_stage(stage, idx).split('/')[-1]+'_1k'+suff # no original training data for nocl
            else:
                fname = fname_fun_stage(stage, idx).split('/')[-1]+suff 

            #load data    
            try:
                dat_stat = retrieve_behdat(fname_behdat,fname)
            except:
                print('Json error. skipping')
                dat_stat = None
            
            if dat_stat is None:
                print('json error or ot added to beh yet:' + fname)
                stagelist.append(stage)
                idxlist.append(idx)
                sensitivity.append(np.nan)
                sensitivity_pval.append(np.nan)
                vio.append(np.nan)
                optout.append(np.nan)
                stagelist_block.append(stage)
                idxlist_block.append(idx)
                histreg_coeff.append(np.nan*np.ones((9,))) #blockdat or regular? 
                histreg_pval.append(np.nan*np.ones((9,)))  #TODO
                rs_hist.append(np.nan)
                wt_ratio.append(np.nan) 
                continue

            # run analysis. normalize w.t. to 20ul for regressions first, avoid MAT if not in block stage
            ops = utils.opsbase()  #shouldnt need specific ops, just dt and stuff
            
            # set necessary ops
            ops['displevel'] = 3
            pc_old = ops['pcatch']
            if stage == 5:
                ops['useblocks'] = True
                ops['pcatch'] = pc_old
            elif stage == 3:
                ops['pcatch'] = 0
                ops['useblocks'] = False
            else:
                ops['useblocks'] = False
                ops['pcatch'] = pc_old
            # analyze       
            try:       
                wt_dict, linreg, numdict, rho_all, ops = wta.parsesimulation(dat_stat, ops, allreg=True)       
            except ValueError:
                print('issue with parsesimulation(): '+fname)
                
            if stage == 5:
                try:
                    #normalize wait times in regression to 20uL
                    #dat_stat['wt'] = dat_stat['wt']/(wt_dict['wt_mixed'][2]/ops['dt'])
                    #or z-score
                    dat_stat['wt'] = (dat_stat['wt']-np.nanmean(dat_stat['wt']))/np.nanstd(dat_stat['wt'])
                    
                    wtlist, reg_hist, reg_block = wta.modelagnostic_test(dat_stat, dt=ops['dt'])
                except ValueError:
                    print('encountered an immature network that cant do MAT with inferred block wait times')
                    #create fake block and regression structs that play nice with later math
                    histreg_coeff = list(np.nan*np.ones((9,))) #blockdat or regular? 
                    histreg_pval = list(np.nan*np.ones((9,)))  #TODO
                    rs_hist = np.nan         
                    reg_block = [histreg_coeff, rs_hist, histreg_pval]
                    wtlist = None
                    reg_hist = None  
            else:
                wtlist = None
                reg_hist = None
                reg_block = None

            #grab mixed slopes from block stage, all trial slope from catch, nocatch
            if fname.find('block') == -1:
                slope = linreg['m']
                pval = linreg['p']
            else:
                slope = linreg['m_mixed']
                pval = linreg['p_mixed']

            nums = numdict
            reg_blockdat = reg_block

            #append to save lists
            stagelist.append(stage)
            idxlist.append(idx)
            sensitivity.append(slope)
            sensitivity_pval.append(pval)
            vio.append(numdict['vio'])
            optout.append(numdict['optout'])
            stagelist_block.append(stage)
            idxlist_block.append(idx)

            # check block coefficient and wt ratio
            if stage > 4:                
                histreg_coeff.append(reg_block[0]) #blockdat or regular? 
                histreg_pval.append(reg_block[2])  #TODO
                rs_hist.append(reg_block[1])

                adapt_20_ratio = wt_dict['wt_high'][2] / wt_dict['wt_low'][2]  
                wt_ratio.append(adapt_20_ratio) 
            else:
                histreg_coeff.append(np.nan*np.ones((9,))) #blockdat or regular? 
                histreg_pval.append(np.nan*np.ones((9,)))  #TODO
                rs_hist.append(np.nan)
                wt_ratio.append(np.nan) 
              

    d = {'sensitivity':sensitivity,
    'sensitivity_pval': sensitivity_pval,
    'optout':optout,
    'vio': vio,
    'wt_ratio':wt_ratio,
    'histreg_coeff':histreg_coeff,
    'histreg_pval':histreg_pval,
    'rs_hist': rs_hist}

                
    return d, stagelist, idxlist, stagelist_block, idxlist_block
                

In [50]:

def acrosstraining_dat_allnet(s_idx = 0, switchblock = False):

    #get across-training behavioral data, averaged across all networks
    
    lenlist = []
    slopelist = []
    optoutlist = []
    adaptationlist = []
    stagelist_list= []
    block_coeff_list = []
    block_coeffcross_list = []
    
    losslist_L = []
    losslist_pi =[]
    losslist_V = []
    losslist_ent = []
    losslist_inf = []
    losslist_kind = []
    lenlist_losses = []
    
    
    print(s_idx)
    
    n_rnn = 0
    n_max = num_rnns[s_idx]

    for j in range(1,n_max+1):

        nnum = j
        print(nnum)
        
        #check if behavioral file exists. if not skip that RNN and notify
        if s_idx ==2 or s_idx == 7 or s_idx == 8:
            fname_behdat = datadir_dat(s_idx)+'rnn_'+str(nnum)+'_allbeh_1k.json'
        else:
            fname_behdat = datadir_dat(s_idx)+'rnn_'+str(nnum)+'_allbeh.json'
          

        if exists(fname_behdat): 
            d, stagelist, idxlist, stagelist_block, idxlist_block = beh_acrosstraining(num = nnum, s_idx = s_idx, 
                                                                                   switchblock = switchblock)
            n_rnn +=1
        else: 
            print('no behavioral file found. skipping this RNN: '+fname_behdat)
            continue

        nsamps = len(d['wt_ratio']) #how many sessions/ data points for that RNN

        #grab items of interest, 
        slopelist.append(np.array(d['sensitivity']))
        optoutlist.append(np.array(d['optout']))
        adaptationlist.append(np.array(d['wt_ratio']))

        block_coeff = [k[-1] for k in d['histreg_coeff']]
        #block_coeff_sig = np.array(block_coeff_pval).astype(int)*np.array(block_coeff)
        block_coeff_sig = np.array(block_coeff)
        block_coeff_list.append(block_coeff_sig)

        #keepping track of stage
        lenlist.append(nsamps)
        stagelist_list.append(stagelist)


    #post-process data. 
    # - pad RNNs that didn't finish training
    # - set reward sensitivity to 0 if stage < 4
    # - set adaptation to 0 if stage < 5

    #pad predlist with nans
    lmax = max(lenlist)
    slopelist_padded = np.nan*np.ones((n_rnn,lmax))
    optoutlist_padded = np.nan*np.ones((n_rnn,lmax))
    ratiolist_padded = np.nan*np.ones((n_rnn,lmax))
    block_coeff_list_padded = np.nan*np.ones((n_rnn,lmax))
    for j in range(n_rnn):
        lj = lenlist[j]
        slopelist_padded[j,:lj] = np.squeeze(slopelist[j])
        optoutlist_padded[j,:lj] = np.squeeze(optoutlist[j])
        ratiolist_padded[j,:lj] = np.squeeze(adaptationlist[j])
        block_coeff_list_padded[j,:lj] = np.squeeze(block_coeff_list[j])
    

    #lenlist[5]
    #len(slopelist[5])

    return lenlist, slopelist_padded, optoutlist_padded, ratiolist_padded, stagelist_list, block_coeff_list_padded



In [51]:
srun = 13
simulate = True
if simulate:
    lenlist, slopelist, optoutlist, ratiolist, stagelist_list, block_coeff_list = acrosstraining_dat_allnet(s_idx = srun)
else:
    print('skipping simulation')


13
1
no behavioral file found. skipping this RNN: /scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/rnn_1_allbeh.json
2
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/rnn_2_allbeh.json
[0, 0, 0, 10, 10, 20]
json error or ot added to beh yet:rnn_curric_2_nocatch_1.json
json error or ot added to beh yet:rnn_curric_2_nocatch_2.json
json error or ot added to beh yet:rnn_curric_2_nocatch_3.json
json error or ot added to beh yet:rnn_curric_2_nocatch_4.json
json error or ot added to beh yet:rnn_curric_2_nocatch_5.json
json error or ot added to beh yet:rnn_curric_2_nocatch_6.json
json error or ot added to beh yet:rnn_curric_2_nocatch_7.json
json error or ot added to beh yet:rnn_curric_2_nocatch_8.json
json error or ot added to beh yet:rnn_curric_2_nocatch_9.json
json error or ot added to beh yet:rnn_curric_2_nocatch_10.json
json error or ot added to beh yet:rnn_curric_2_catch_1.json
json error or ot added to beh yet:rnn_curric_2_catch_2.json
json error or ot adde

json error or ot added to beh yet:rnn_curric_5_block_2.json
json error or ot added to beh yet:rnn_curric_5_block_3.json
json error or ot added to beh yet:rnn_curric_5_block_4.json
json error or ot added to beh yet:rnn_curric_5_block_5.json
json error or ot added to beh yet:rnn_curric_5_block_6.json
json error or ot added to beh yet:rnn_curric_5_block_7.json
json error or ot added to beh yet:rnn_curric_5_block_8.json
json error or ot added to beh yet:rnn_curric_5_block_9.json
json error or ot added to beh yet:rnn_curric_5_block_10.json
json error or ot added to beh yet:rnn_curric_5_block_11.json
json error or ot added to beh yet:rnn_curric_5_block_12.json
json error or ot added to beh yet:rnn_curric_5_block_13.json
json error or ot added to beh yet:rnn_curric_5_block_14.json
json error or ot added to beh yet:rnn_curric_5_block_15.json
json error or ot added to beh yet:rnn_curric_5_block_16.json
json error or ot added to beh yet:rnn_curric_5_block_17.json
json error or ot added to beh ye

95, 20]
/scratch/dh148/dynamics/results/rnn/ac/20231003/gen_kind_newreg/rnn_9_allbeh.json
[0, 0, 0, 10, 10, 20]
json error or ot added to beh yet:rnn_curric_9_nocatch_1.json
json error or ot added to beh yet:rnn_curric_9_nocatch_2.json
json error or ot added to beh yet:rnn_curric_9_nocatch_3.json
json error or ot added to beh yet:rnn_curric_9_nocatch_4.json
json error or ot added to beh yet:rnn_curric_9_nocatch_5.json
json error or ot added to beh yet:rnn_curric_9_nocatch_6.json
json error or ot added to beh yet:rnn_curric_9_nocatch_7.json
json error or ot added to beh yet:rnn_curric_9_nocatch_8.json
json error or ot added to beh yet:rnn_curric_9_nocatch_9.json
json error or ot added to beh yet:rnn_curric_9_nocatch_10.json
json error or ot added to beh yet:rnn_curric_9_catch_1.json
json error or ot added to beh yet:rnn_curric_9_catch_2.json
json error or ot added to beh yet:rnn_curric_9_catch_3.json
json error or ot added to beh yet:rnn_curric_9_catch_4.json
json error or ot added to b

In [53]:
# save
savename = savedir + 'behdat_sidx_'+str(srun)+'.dat'
with open(savename,'wb') as f:
    pickle.dump([lenlist, slopelist, optoutlist, ratiolist, stagelist_list, block_coeff_list],f)